In [2]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
import fred_msa

In [3]:
api_key = 'a37b50cd27afbc3ce23a81ddc5541dec'
endpoint = 'https://api.stlouisfed.org/fred/series/categories'

In [5]:
state_ids = fred_msa.get_state_ids(api_key)

In [6]:
state_msas = {}
for state, state_id in state_ids.items():
    state_msas[state] = fred_msa.get_msa_cats(api_key, state_id)
    time.sleep(5)

In [7]:
series = pd.DataFrame()

In [10]:
for state, msas in state_msas.items():
    for msa, msa_id in msas.items():
        msa_series = fred_msa.get_msa_series_list(msa_id, api_key)
        msa_series['state'] = state
        msa_series['msa'] = msa
        
        series = series.append(msa_series)
        time.sleep(3)

In [12]:
series.to_csv('..\\fred-data\msa_series.csv', index=False)